In [1]:
import os
import stat
import time
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.document_loaders import DirectoryLoader
from langchain_community.document_loaders import TextLoader
from langchain_community.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.text_splitter import (
    CharacterTextSplitter,
)
from langchain.prompts.chat import (
    HumanMessagePromptTemplate,
    SystemMessagePromptTemplate,
)
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema.runnable import RunnablePassthrough
from langchain_community.vectorstores import Chroma
from colorama import Fore
import warnings

warnings.filterwarnings("ignore")

load_dotenv()

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
LANGUAGE_MODEL = "gpt-3.5-turbo-instruct"


In [2]:
template: str = """/
    تو یک مفسر مثنوی مولانا هستی. سعی کن جواب سوال : /
      {question} /
      :رو بر اساس منبع روبرو /
      {context}
    بدهی. /
    """

system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_message_prompt = HumanMessagePromptTemplate.from_template(
    input_variables=["question", "context"],
    template="{question}",
)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

model = ChatOpenAI()


In [3]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])



def load_documents():
    """Load all text files from a directory, split them into chunks, and return the split documents."""
    loader = DirectoryLoader("./output_text/", glob="*.txt")  # Load all .txt files
    raw_documents = loader.load()

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=100,
        chunk_overlap=50,
        separators=["\n\n", "\n", " ", ""]
    )
    return text_splitter.split_documents(raw_documents)



def load_embeddings(documents, user_query):
    """Create or load a vector store from a set of documents."""
    persist_directory = './chroma_cache'  # Directory to store embeddings
    embedding_model = OpenAIEmbeddings()

    # Ensure the directory exists and has write permissions
    if not os.path.exists(persist_directory):
        os.makedirs(persist_directory, exist_ok=True)
    else:
        # Check if the directory is writable
        if not os.access(persist_directory, os.W_OK):
            print(f"Error: No write access to {persist_directory}. Fixing permissions...")
            try:
                os.chmod(persist_directory, stat.S_IWUSR | stat.S_IRUSR | stat.S_IXUSR)
            except Exception as e:
                print(f"Failed to change directory permissions: {e}")
                return None

    try:
        # Load or create Chroma vector store
        if not os.listdir(persist_directory):  # Empty directory means no existing DB
            print("Initializing new ChromaDB instance...")
            db = Chroma.from_documents(documents, embedding_model, persist_directory=persist_directory)
            db.persist()
        else:
            print("Loading existing ChromaDB instance...")
            db = Chroma(persist_directory=persist_directory, embedding_function=embedding_model)

        # Perform similarity search
        docs = db.similarity_search(user_query, k=20)
        print("\nRetrieved Documents:\n")
        print(format_docs(docs))  # Print the retrieved documents
        return db.as_retriever()

    except Exception as e:
        print(f"Error while loading ChromaDB: {e}")
        return None


def generate_response(retriever, query):
    pass
    # Create a prompt template using a template from the config module and input variables
    # representing the context and question.
    # create the prompt

    chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | chat_prompt_template
        | model
        | StrOutputParser()
    )
    return chain.invoke(query)

def query(query):
    documents = load_documents()
    retriever = load_embeddings(documents, query)
    response = generate_response(retriever, query)
    return response

In [4]:
documents = load_documents()

In [5]:
retriever = load_embeddings(documents, """تفسیر بیت زیر چیست: /"
"یاد من کن پیش تخت آن عزیز /
تا مرا هم واخرد زین حبس نیز""")

Initializing new ChromaDB instance...

Retrieved Documents:

شاه یاد من کن پیش تخت آن عزیز. تا مرا هم واخرد زین حبس نیز. و مولوی اینجا را اشاره می کنیم. کی دهد

رو بیشتر یاد رفتن بیاندازه نه یاد موندن، یاد زوال بیاندازه که همه چیز رفت حتی الان زیر خاک سردی

از آن سو از منعرهٔ من صدای من رو بشنوی، اینکه رسم گفتبانی،گوی عاشقان نشو. بعد میاد سراب گفتگوی

یادآرچون فرو گیرت غمت گر چستهای زن دم نومید کن وا جستهای اگر غمی سراغ تو آمد، اگر شخص زیرکی باشی،

و به یاد ما می آورد که ناامیری کجا پیش میاد کجا پیشینه میادو در اینجا نسبت عقل با عشق چیست نسبت عشق

باشی. ترس، به تعبیر من شرم، مرحله دوم و پس از او، ورود مرحله عشق و عرفان.ظهد اندر کاشتن کوشیدن است،

از سر من برمدار بیقرارم بیقرارم بیقرارم خوابها بیزار شد از چشم من در غمت ایرش کسروی آسمنگر نیام

بیقرار.خوابها بیزار شد از چشم من بر غمت، بیرش که سر بوم بی آسمت.گر نیام لایق چه باشد گر دمی ناسزایی

مرتبه یاد خود افتاده، مرتبه به یاد خود افتاده، که در کنان دام، در دام عشقفت، در دام هجران افتاده،

از خود اشتحی بشه، یاد گرفت که در پای معشوق خضوع 

In [7]:
response = generate_response(retriever, """تفسیر بیت زیر چیست: /"
"یاد من کن پیش تخت آن عزیز /
تا مرا هم واخرد زین حبس نیز""")

print(response)

بیت "یاد من کن پیش تخت آن عزیز / تا مرا هم واخرد زین حبس نیز" به معنای "به یاد من باش و در حضور آن شاه عزیز خود را به آزادی از این زندان فکر کن" است. در این بیت، شاعر به شکنجه و زندانی بودن خود اشاره می‌کند و از شاه می‌خواهد که به یادش باشد و او را از این وضعیت بیرون بیاورد. این بیت می‌تواند به شکل مجازی نمادی از زندگی معنوی و تحول روحی باشد، جایی که فرد به دنبال آزادی از بندگی نفس و هواهای زمینی است.
